# 0. 전체 Notebook 코드 (VSCode/Jupyter에서 바로 쓰기 좋은 형태)

In [67]:
# %% [markdown]
# # pepbind_pipeline 단계별 디버그 노트북
# - 기존 run 결과 폴더(pipeline/RUN_ID)를 기준으로
# - ColabFold → Prodigy → Vina → PLIP → (최종 스코어) 를
#   단계별로 따로 실행/재실행하기 위한 노트북 템플릿입니다.
# %%
from pathlib import Path
import importlib

import pepbind_pipeline as pb  # pepbind_pipeline.py 가 이 노트북과 같은 프로젝트 안에 있다고 가정

# pepbind_pipeline 수정 후에는 이 셀을 다시 실행해서 최신 코드로 reload
pb = importlib.reload(pb)

# 디버그하고 싶은 run ID (폴더 이름)만 바꿔서 사용
RUN_ID = "PDP_20251123_225850"  # ← 여기만 바꿔주면 됨

[INFO] PyTorch device: cuda


In [68]:
# %%
# 기본 경로 탐색 유틸 (pipeline/RUN_ID 를 현재 작업 디렉토리 주변에서 자동 탐색)

def find_run_dir(run_id: str) -> Path:
    cwd = Path.cwd().resolve()

    candidate_dirs = [
        # pepbind_pipeline.py 에서 쓰는 BASE_DIR 우선
        getattr(pb, "BASE_DIR", Path("~/work/pipeline").expanduser()) / run_id,

        # 기존 후보들
        cwd / "pipeline" / run_id,
        cwd.parent / "pipeline" / run_id,
        cwd.parent.parent / "pipeline" / run_id,
        cwd / run_id,
    ]

    checked = []
    print("===== RUN_DIR 후보 경로 체크 =====")
    for p in candidate_dirs:
        p = p.resolve()
        if p in checked:
            continue
        checked.append(p)
        print(f"{p} : {'OK' if p.is_dir() else 'NOT FOUND'}")
        if p.is_dir():
            return p

    raise FileNotFoundError(
        f"RUN_ID={run_id} 에 해당하는 폴더를 찾지 못했습니다.\n"
        f"위에 출력된 경로를 보고, 실제 위치를 확인한 뒤 아래에서 RUN_DIR 를 수동으로 지정해 주세요."
    )

try:
    RUN_DIR = find_run_dir(RUN_ID)
except FileNotFoundError as e:
    print("\n자동 탐색 실패. 필요하면 아래 주석을 풀고 수동 지정하세요.")
    # 예시:
    # RUN_DIR = Path("/home/aisys/work/pipeline") / RUN_ID
    raise

print("\n===== 사용 RUN_DIR =====")
print(RUN_DIR)

===== RUN_DIR 후보 경로 체크 =====
/home/aisys/work/pipeline/PDP_20251123_225850 : OK

===== 사용 RUN_DIR =====
/home/aisys/work/pipeline/PDP_20251123_225850


In [69]:
# %%
# 서브 디렉토리 경로 설정

FASTA_DIR      = RUN_DIR / "fasta"
PDB_DIR        = RUN_DIR / "pdb"
COLABFOLD_DIR  = PDB_DIR / "colabfold_output"
RESULTS_DIR    = RUN_DIR / "results"
TEMP_DIR       = RUN_DIR / "temp"

for path in [RUN_DIR, FASTA_DIR, PDB_DIR, COLABFOLD_DIR, RESULTS_DIR, TEMP_DIR]:
    print(f"{path} : {'OK' if path.exists() else 'MISSING'}")

/home/aisys/work/pipeline/PDP_20251123_225850 : OK
/home/aisys/work/pipeline/PDP_20251123_225850/fasta : OK
/home/aisys/work/pipeline/PDP_20251123_225850/pdb : OK
/home/aisys/work/pipeline/PDP_20251123_225850/pdb/colabfold_output : OK
/home/aisys/work/pipeline/PDP_20251123_225850/results : OK
/home/aisys/work/pipeline/PDP_20251123_225850/temp : OK


In [70]:
# %%
# 기본 상태 확인용 헬퍼 (중간 중간 상태 보고 싶을 때 호출)

def show_basic_status():
    print("===== 기본 경로 상태 =====")
    print("RUN_DIR      :", RUN_DIR, "OK" if RUN_DIR.exists() else "MISSING")
    print("FASTA_DIR    :", FASTA_DIR, "OK" if FASTA_DIR.exists() else "MISSING")
    print("PDB_DIR      :", PDB_DIR, "OK" if PDB_DIR.exists() else "MISSING")
    print("COLABFOLD_DIR:", COLABFOLD_DIR, "OK" if COLABFOLD_DIR.exists() else "MISSING")
    print("RESULTS_DIR  :", RESULTS_DIR, "OK" if RESULTS_DIR.exists() else "MISSING")
    print("TEMP_DIR     :", TEMP_DIR, "OK" if TEMP_DIR.exists() else "MISSING")

    if COLABFOLD_DIR.exists():
        pdb_list = list(COLABFOLD_DIR.glob("*.pdb"))
        print("  └─ ColabFold PDB 개수:", len(pdb_list))
    else:
        print("  └─ ColabFold PDB 없음")

show_basic_status()

===== 기본 경로 상태 =====
RUN_DIR      : /home/aisys/work/pipeline/PDP_20251123_225850 OK
FASTA_DIR    : /home/aisys/work/pipeline/PDP_20251123_225850/fasta OK
PDB_DIR      : /home/aisys/work/pipeline/PDP_20251123_225850/pdb OK
COLABFOLD_DIR: /home/aisys/work/pipeline/PDP_20251123_225850/pdb/colabfold_output OK
RESULTS_DIR  : /home/aisys/work/pipeline/PDP_20251123_225850/results OK
TEMP_DIR     : /home/aisys/work/pipeline/PDP_20251123_225850/temp OK
  └─ ColabFold PDB 개수: 0


# 1. pepbind_pipeline 안에 어떤 함수들이 있는지 빠르게 훑어보기

In [71]:
# %% [markdown]
# ## 1. pepbind_pipeline 안에 어떤 함수들이 있는지 빠르게 훑어보기
# - 이름에 colab / fasta / peptide / prodigy / vina / plip / score 등이 들어간 함수들을 찾아서
#   실제 함수명을 확인합니다.
# - 만약 내가 아래 셀에서 호출한 함수 이름과 조금이라도 다르면
#   이 셀 결과를 보고 함수 이름만 맞게 수정해주면 됩니다.

# %%
keywords = ["colab", "fasta", "peptide", "prodigy", "vina", "plip", "score", "rank"]
for key in keywords:
    names = [n for n in dir(pb) if key.lower() in n.lower()]
    if not names:
        continue
    print(f"\n[{key}] 관련 함수/변수")
    for n in names:
        attr = getattr(pb, n)
        kind = "function" if callable(attr) else type(attr).__name__
        print(f" - {n} ({kind})")


[colab] 관련 함수/변수
 - COLABFOLD_CMD (str)
 - COLABFOLD_CPU_FALLBACK (bool)
 - COLABFOLD_MAX_IDLE_MIN (int)
 - COLABFOLD_MAX_MSA (str)
 - COLABFOLD_MAX_TOTAL_MIN (int)
 - RUN_COLABFOLD (bool)
 - prepare_colabfold_batch_csv (function)
 - run_colabfold_batch_with_progress (function)

[fasta] 관련 함수/변수
 - write_peptide_fasta (function)
 - write_target_fasta (function)

[peptide] 관련 함수/변수
 - NUM_PEPTIDES (int)
 - PEPTIDE_LENGTH (int)
 - generate_peptides_with_mlm (function)
 - write_peptide_fasta (function)

[prodigy] 관련 함수/변수
 - PRODIGY_SCRIPT (str)
 - PRODIGY_SUMMARY_COLS (list)
 - RUN_PRODIGY (bool)
 - load_prodigy_scores (function)
 - parse_prodigy_dg_from_stdout (function)
 - run_prodigy_on_rank1 (function)

[vina] 관련 함수/변수
 - RUN_VINA (bool)
 - VINA_CMD (str)
 - VINA_SUMMARY_COLS (list)
 - load_vina_scores (function)
 - parse_vina_score_from_stdout (function)
 - run_vina_on_rank1 (function)

[plip] 관련 함수/변수
 - PLIP_CMD (str)
 - PLIP_SUMMARY_COLS (list)
 - RUN_PLIP (bool)
 - load_plip_sc

# 2. (선택) FASTA/펩타이드 생성 단계만 따로 돌리고 싶을 때

In [ ]:
from typing import List

# 이 RUN_ID에서 사용할 FASTA 파일 경로
TARGET_FASTA = FASTA_DIR / "target_protein.fasta"
PEPTIDE_FASTA = FASTA_DIR / "peptides.fasta"

def load_target_sequence(fasta_path: Path) -> str:
    """target_protein.fasta 에서 타깃 단백질 서열 읽기 (없으면 pepbind_pipeline.TARGET_SEQUENCE 사용)"""
    if fasta_path.exists():
        seq = []
        with open(fasta_path) as f:
            for line in f:
                line = line.strip()
                if (not line) or line.startswith(">"):
                    continue
                seq.append(line)
        s = "".join(seq).strip()
        print(f"{fasta_path.name} 에서 타깃 서열 로드 (길이 {len(s)})")
        return s

    # FASTA 파일이 없다면 pepbind_pipeline 안의 TARGET_SEQUENCE를 사용
    if hasattr(pb, "TARGET_SEQUENCE"):
        s = str(pb.TARGET_SEQUENCE).strip()
        print(f"{fasta_path.name} 없음 → pepbind_pipeline.TARGET_SEQUENCE 사용 (길이 {len(s)})")
        return s

    raise FileNotFoundError(
        f"{fasta_path} 가 없고 pepbind_pipeline.TARGET_SEQUENCE 도 없습니다."
    )

def load_peptides_from_fasta(fasta_path: Path) -> List[str]:
    """peptides.fasta 에서 펩타이드 서열 리스트 읽기"""
    peptides = []
    seq = ""
    with open(fasta_path) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if seq:
                    peptides.append(seq)
                    seq = ""
            else:
                seq += line
        if seq:
            peptides.append(seq)
    print(f"{fasta_path.name} 에서 펩타이드 {len(peptides)}개 로드")
    return peptides

# 1) 타깃 서열 로드 (파일 있으면 파일, 없으면 pepbind_pipeline.TARGET_SEQUENCE)
TARGET_SEQ = load_target_sequence(TARGET_FASTA)

# 2) 펩타이드 목록: 보통은 이미 pepbind_pipeline 실행 당시 생성되어 있음
if PEPTIDE_FASTA.exists():
    PEPTIDES = load_peptides_from_fasta(PEPTIDE_FASTA)
else:
    print("peptides.fasta 가 없어 새로 생성합니다. (ESM-2 모델 사용)")

    tokenizer, model = pb.load_esm_mlm()
    PEPTIDES = pb.generate_peptides_with_mlm(
        tokenizer,
        model,
        TARGET_SEQ,
        num_peptides=getattr(pb, "NUM_PEPTIDES", 10),
        peptide_len=getattr(pb, "PEPTIDE_LENGTH", 4),
        top_k=getattr(pb, "PEPMLM_TOP_K", 10),
        temperature=getattr(pb, "PEPMLM_TEMPERATURE", 1.0),
    )

    # FASTA 폴더 보장 후 저장
    FASTA_DIR.mkdir(parents=True, exist_ok=True)
    pb.write_target_fasta(FASTA_DIR, TARGET_SEQ)
    pb.write_peptide_fasta(FASTA_DIR, PEPTIDES)
    print("target_protein.fasta / peptides.fasta 새로 저장 완료.")

print("\n[요약]")
print(" - 타깃 서열 길이:", len(TARGET_SEQ))
print(" - 펩타이드 개수 :", len(PEPTIDES))

target_protein.fasta 에서 타깃 서열 로드 (길이 222)
peptides.fasta 에서 펩타이드 10개 로드

[요약]
 - 타깃 서열 길이: 222
 - 펩타이드 개수 : 10


# 3. (선택) ColabFold 단계만 따로 실행

In [73]:
# ColabFold 단계만 다시 실행
# (GPU → OOM 나면 pepbind_pipeline.run_colabfold_batch_with_progress 안에서 CPU fallback 시도)

# 2번 셀에서 TARGET_SEQ, PEPTIDES 를 준비했는지 확인
if "TARGET_SEQ" not in globals() or "PEPTIDES" not in globals():
    raise RuntimeError("먼저 2번 셀(FASTA/펩타이드 로드)을 실행해서 TARGET_SEQ, PEPTIDES 변수를 준비해 주세요.")

if not PEPTIDES:
    raise RuntimeError("PEPTIDES 리스트가 비어 있습니다. peptides.fasta 내용을 확인해 주세요.")

print("\n[ColabFold 입력 준비]")
print(" - 타깃 서열 길이:", len(TARGET_SEQ))
print(" - 펩타이드 개수 :", len(PEPTIDES))

# ColabFold 배치용 CSV 생성 (기존 pepbind_pipeline 함수 그대로 사용)
COLAB_CSV = pb.prepare_colabfold_batch_csv(
    TEMP_DIR,
    TARGET_SEQ,
    PEPTIDES,
)
print(" - CSV 경로:", COLAB_CSV)

# 필요하면 이전 ColabFold 결과를 싹 지우고 깨끗하게 시작하고 싶을 때:
# (주석을 풀면 기존 rank_001 PDB, json, npz 등이 전부 삭제되니 조심!)
# import shutil
# if COLABFOLD_DIR.exists():
#     for p in COLABFOLD_DIR.glob("*"):
#         if p.is_file():
#             p.unlink()
#         else:
#             shutil.rmtree(p)
#     print("이전 ColabFold 출력 삭제 완료.")

print("\n[ColabFold 실행 시작]")
rank1_pdbs = pb.run_colabfold_batch_with_progress(
    COLAB_CSV,
    COLABFOLD_DIR,
    total_complexes=len(PEPTIDES),
)

print("\n[ColabFold 결과 요약]")
print("rank_001 PDB 총 개수:", len(rank1_pdbs))
for p in rank1_pdbs[:5]:
    print(" -", p.name)


[ColabFold 입력 준비]
 - 타깃 서열 길이: 222
 - 펩타이드 개수 : 10
✅ ColabFold 배치 CSV 생성 (id,sequence 형식): /home/aisys/work/pipeline/PDP_20251123_225850/temp/batch_complexes.csv
 - CSV 경로: /home/aisys/work/pipeline/PDP_20251123_225850/temp/batch_complexes.csv

[ColabFold 실행 시작]

STEP 3: ColabFold 배치 실행
[INFO] 실행 명령어:
  colabfold_batch --num-recycle 1 --model-type alphafold2_multimer_v3 --rank ptm --max-msa 16:64 --num-models 1 --stop-at-score 0.5 /home/aisys/work/pipeline/PDP_20251123_225850/temp/batch_complexes.csv /home/aisys/work/pipeline/PDP_20251123_225850/pdb/colabfold_output
[INFO] GPU 메모리 초기화: torch.cuda.empty_cache() 실행
[INFO] Python GC 실행 완료
[INFO] ColabFold GPU 모드 실행
[INFO] 로그 파일: /home/aisys/work/pipeline/PDP_20251123_225850/pdb/colabfold_output/colabfold_batch.log
[ColabFold 진행 상황(GPU)] 0/10 구조 완료
[ERROR] ColabFold(GPU) 실행 실패 (returncode=1). 마지막 40줄 로그:
Traceback (most recent call last):
  File "/home/aisys/miniconda3/envs/pepbind/bin/colabfold_batch", line 7, in <module>
    sys.exit(ma

# 4. ColabFold 결과에서 rank_001 PDB만 모으기

In [74]:
# %% [markdown]
# ## 4. ColabFold 결과에서 rank_001 PDB만 모으기
# - 기존 pepbind_pipeline.py 에 collect_rank1_pdbs 같은 함수가 있다면 그대로 사용
# - 없다면 노트북 안에서 간단히 glob 패턴으로 구현

# %%
from typing import List

if hasattr(pb, "collect_rank1_pdbs"):
    collect_rank1_pdbs = pb.collect_rank1_pdbs  # 기존 함수 재사용
    print("pepbind_pipeline.collect_rank1_pdbs 함수를 사용합니다.")
else:
    def collect_rank1_pdbs(colabfold_dir: Path) -> List[Path]:
        pattern = "*rank_001*.*pdb"
        pdbs = sorted(colabfold_dir.glob(pattern))
        print(f"pattern='{pattern}' 로 찾은 rank_001 PDB 개수: {len(pdbs)}")
        return pdbs

if not COLABFOLD_DIR.exists():
    raise RuntimeError("COLABFOLD_DIR 가 존재하지 않습니다. ColabFold 결과가 있는 run ID 인지 확인하세요.")

rank1_pdbs = collect_rank1_pdbs(COLABFOLD_DIR)
print(f"\nrank_001 PDB 총 개수: {len(rank1_pdbs)}")
for p in rank1_pdbs[:5]:
    print(" -", p.name)

pattern='*rank_001*.*pdb' 로 찾은 rank_001 PDB 개수: 10

rank_001 PDB 총 개수: 10
 - complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_3_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_4_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb


# 5. Vina 단계만 따로 실행

In [75]:
# %% [markdown]
# ## 6. Vina 단계만 따로 실행
# - 방금 수정한 run_vina_on_rank1(rank1_pdbs, vina_dir: Path)를 그대로 사용
# - 이미 rank_001 PDB 리스트를 확보했으므로, 여기서는 ColabFold를 다시 돌릴 필요가 없음
# - Vina 관련 코드만 수정하고 싶을 때 이 셀만 계속 돌리면 됨

# %%
VINA_DIR = RESULTS_DIR / "vina"
VINA_DIR.mkdir(parents=True, exist_ok=True)
print("VINA_DIR:", VINA_DIR)

# pepbind_pipeline.py 에 들어 있는 run_vina_on_rank1 함수를 그대로 사용
pb.run_vina_on_rank1(rank1_pdbs, VINA_DIR)

vina_summary = VINA_DIR / "vina_summary.xlsx"
if vina_summary.exists():
    print("✅ vina_summary.xlsx 생성 완료:", vina_summary)
else:
    print("⚠ vina_summary.xlsx 를 찾지 못했습니다. 함수 내부에서 파일 이름을 확인해 보세요.")

VINA_DIR: /home/aisys/work/pipeline/PDP_20251123_225850/results/vina

STEP 4: AutoDock Vina 도킹

[INFO] Vina 준비: complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
[INFO] complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb 체인 구성: {'A': 222, 'B': 4}
[INFO] 자동 할당 체인: receptor=A, ligand=B
[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251123_225850/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb -xr -opdbqt -O /home/aisys/work/pipeline/PDP_20251123_225850/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdbqt
[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251123_225850/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_m

In [76]:
# rank_001 PDB 중 하나 골라서 체인 구성 확인해보기

if not rank1_pdbs:
    raise RuntimeError("rank_001 PDB가 없습니다. ColabFold 결과를 확인하세요.")

sample_pdb = rank1_pdbs[0]  # 필요하면 인덱스 바꿔가면서 확인
print("샘플 PDB:", sample_pdb)

counts = pb.get_chain_residue_counts(sample_pdb)
print("체인별 residue 개수:")
for cid, n in counts.items():
    print(f"  chain {cid!r}: {n} residues")

샘플 PDB: /home/aisys/work/pipeline/PDP_20251123_225850/pdb/colabfold_output/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
체인별 residue 개수:
  chain 'A': 222 residues
  chain 'B': 4 residues


# 6. PLIP 단계만 따로 실행

In [77]:
# %% [markdown]
# ## 7. PLIP 단계만 따로 실행
# - run_plip_on_rank1(rank1_pdbs, plip_dir) 형태를 가정
# - 함수 이름/인자는 pepbind_pipeline.py 에 맞게 필요시 수정

# %%
PLIP_DIR = RESULTS_DIR / "plip"
PLIP_DIR.mkdir(parents=True, exist_ok=True)
print("PLIP_DIR:", PLIP_DIR)

if hasattr(pb, "run_plip_on_rank1"):
    pb.run_plip_on_rank1(rank1_pdbs, PLIP_DIR)
else:
    print("pepbind_pipeline.run_plip_on_rank1 함수를 찾지 못했습니다.")
    print("→ pepbind_pipeline.py 안 PLIP 실행 함수 이름을 확인해서 여기서 직접 호출해 주세요.")

plip_summary = PLIP_DIR / "plip_summary.xlsx"
if plip_summary.exists():
    print("✅ plip_summary.xlsx 생성 완료:", plip_summary)
else:
    print("⚠ plip_summary.xlsx 를 찾지 못했습니다. 함수 내부에서 파일 이름을 확인해 보세요.")

PLIP_DIR: /home/aisys/work/pipeline/PDP_20251123_225850/results/plip

STEP 5: PLIP 상호작용 분석
[RUN] plip -f /home/aisys/work/pipeline/PDP_20251123_225850/pdb/colabfold_output/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb -o /home/aisys/work/pipeline/PDP_20251123_225850/results/plip/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000 -x -t --chains [['A'], ['B']]
✔️ PLIP 완료: complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb → /home/aisys/work/pipeline/PDP_20251123_225850/results/plip/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000
[RUN] plip -f /home/aisys/work/pipeline/PDP_20251123_225850/pdb/colabfold_output/complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb -o /home/aisys/work/pipeline/PDP_20251123_225850/results/plip/complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000 -x -t --chains [['A'], ['B']]
✔️ PLIP 완료: complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_

# 7. Prodigy 단계만 따로 실행

In [78]:
# %% [markdown]
# ## 5. Prodigy 단계만 따로 실행
# - run_prodigy_on_rank1(rank1_pdbs, prodigy_dir) 형태의 함수를 가정
# - pepbind_pipeline.py 에서 함수 이름이 다르면 이 셀에서만 이름 맞게 바꿔주면 됨
# - 이미 Prodigy 결과가 있는 경우에도, 함수 수정 후 이 셀만 다시 돌려서 재계산 가능

# %%
PRODIGY_DIR = RESULTS_DIR / "prodigy"
PRODIGY_DIR.mkdir(parents=True, exist_ok=True)
print("PRODIGY_DIR:", PRODIGY_DIR)

if hasattr(pb, "run_prodigy_on_rank1"):
    pb.run_prodigy_on_rank1(rank1_pdbs, PRODIGY_DIR)
else:
    print("pepbind_pipeline.run_prodigy_on_rank1 함수를 찾지 못했습니다.")
    print("→ pepbind_pipeline.py 안에서 Prodigy 실행하는 함수 이름을 확인해서, 이 셀에서 직접 호출해 주세요.")

prodigy_summary = PRODIGY_DIR / "prodigy_summary.xlsx"
if prodigy_summary.exists():
    print("✅ prodigy_summary.xlsx 생성 완료:", prodigy_summary)
else:
    print("⚠ prodigy_summary.xlsx 를 찾지 못했습니다. 함수 내부에서 파일 이름을 확인해 보세요.")

PRODIGY_DIR: /home/aisys/work/pipeline/PDP_20251123_225850/results/prodigy

STEP 6: PRODIGY 결합 친화도 평가
[RUN] prodigy /home/aisys/work/pipeline/PDP_20251123_225850/pdb/colabfold_output/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb --selection A B
[WARN] PRODIGY 접촉 없음: complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000. (로그: complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_prodigy.txt)
[RUN] prodigy /home/aisys/work/pipeline/PDP_20251123_225850/pdb/colabfold_output/complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb --selection A B
[RUN] prodigy /home/aisys/work/pipeline/PDP_20251123_225850/pdb/colabfold_output/complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb --selection A B
[WARN] PRODIGY 접촉 없음: complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000. (로그: complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_prodigy.txt)
[RUN] prodigy /home/aisys/work

# 8. (선택) 최종 스코어/랭킹 통합 단계

In [79]:
# %% [markdown]
# ## 8. (선택) 최종 스코어/랭킹 통합 단계
# - Prodigy/Vina/PLIP/ColabFold 점수를 종합해서 최종 점수를 계산하는 함수가
#   pepbind_pipeline.py 안에 있다면 그 함수를 그대로 호출하면 됩니다.
# - 함수 이름을 모르겠다면, 1번 셀(함수 리스트 출력)을 참고해서 수정하세요.

# %%
def load_peptides_from_fasta(fasta_path: Path):
    """peptides.fasta 에서 펩타이드 서열 리스트만 뽑아오기 (간단 버전)"""
    peptides = []
    seq = ""
    with open(fasta_path) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if seq:
                    peptides.append(seq)
                    seq = ""
            else:
                seq += line
        if seq:
            peptides.append(seq)
    return peptides

# pepbind_pipeline.py 의 init_workspace() 에서 쓰는 키 구조에 맞게 폴더 dict 구성
FOLDERS = {
    "root": RUN_DIR,
    "fasta": FASTA_DIR,
    "pdb": PDB_DIR,
    "colabfold_out": COLABFOLD_DIR,
    "results": RESULTS_DIR,
    "vina": RESULTS_DIR / "vina",
    "plip": RESULTS_DIR / "plip",
    "prodigy": RESULTS_DIR / "prodigy",
    "temp": TEMP_DIR,
}

PEPTIDES = load_peptides_from_fasta(FASTA_DIR / "peptides.fasta")

final_xlsx = pb.build_and_save_final_table(FOLDERS, PEPTIDES, rank1_pdbs)
print("최종 결과 엑셀:", final_xlsx)

print("\nRESULTS_DIR 안의 엑셀 파일 확인:")
for xlsx_path in sorted(RESULTS_DIR.glob("*.xlsx")):
    print(" -", xlsx_path.name)

[INFO] Vina 요약 엑셀에서 점수 로드: /home/aisys/work/pipeline/PDP_20251123_225850/results/vina/vina_summary.xlsx
[INFO] Vina 점수를 읽어온 구조 수: 10
[INFO] PRODIGY 요약 엑셀에서 점수 로드: /home/aisys/work/pipeline/PDP_20251123_225850/results/prodigy/prodigy_summary.xlsx
[INFO] PRODIGY ΔG를 요약 파일에서 불러옴: 10개 구조
[INFO] ipTM 값을 읽어온 구조 수: 10 / 10
[INFO] PLIP 파싱 디버그 로그: /home/aisys/work/pipeline/PDP_20251123_225850/results/plip/plip_parse_debug.txt
[INFO] PLIP 요약 엑셀 저장: /home/aisys/work/pipeline/PDP_20251123_225850/results/plip/plip_summary.xlsx
[INFO] PLIP 상호작용을 읽어온 구조 수: 10
✅ 최종 결과 엑셀 저장: /home/aisys/work/pipeline/PDP_20251123_225850/results/final_peptide_rank_20251124_161913.xlsx
최종 결과 엑셀: /home/aisys/work/pipeline/PDP_20251123_225850/results/final_peptide_rank_20251124_161913.xlsx

RESULTS_DIR 안의 엑셀 파일 확인:
 - final_peptide_rank_20251124_161913.xlsx
